In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetSimCLR(nn.Module):

    def __init__(self, device="cpu", embedding_dim=64):
        super(ResNetSimCLR, self).__init__()
        # The embedding_dim is the dimension of the final output dimension
        self.embedding_dim = embedding_dim
        # initiate the resnet backbone
        self.model = models.resnet50(weights=None)
        # dimension of the final CNN layer in resnet (2048 normally)
        self.dim_mlp = self.model.fc.in_features
        # FC network with one hidden layer added on top of the ResNet
        self.model.fc = torch.nn.Sequential(torch.nn.Linear(self.dim_mlp, self.dim_mlp),
                                       torch.nn.ReLU(),
                                       torch.nn.Linear(self.dim_mlp, self.embedding_dim),
                                       torch.nn.ReLU())

    def forward(self, x):
        return self.model(x)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNetSimCLR(device)

In [2]:
model

ResNetSimCLR(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

MEAN=[0.4914, 0.4822, 0.4465]
STD=[0.2470, 0.2435, 0.2616]

img_size = 32
batch_size = 32
root_folder = "./dataset/data"

# trasnformations for the dataset
transform = transforms.Compose([
                transforms.RandomResizedCrop(img_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=MEAN, std=STD)
            ])

# initiate the dataset
dataset = CIFAR10(root=root_folder, train=True, download=True, transform=transform)
#select 1000 sample
subset_indices = list(range(1000))
small_dataset = torch.utils.data.Subset(dataset, subset_indices)
# batching of the dataset
data_loader = DataLoader(small_dataset, batch_size=batch_size, shuffle=True)
# initiate the validation dataset
dataset_val = CIFAR10(root=root_folder, train=False, download=True, transform=transform)
#select 200 validation sample
subset_indices_val = list(range(200))
small_dataset_val = torch.utils.data.Subset(dataset_val, subset_indices_val)
# DataLoader for batching
data_loader_val = DataLoader(small_dataset_val, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from dataset import preprocess_for_train

In [5]:
from tqdm import tqdm
import numpy as np

def train_one_epoch(model, dataloader, device, img_size, normalizer, criterion, optimizer, epoch, tracker, scheduler = None):
    model.train()
    losses = []
    for mini_batch in tqdm(dataloader):
        mini_batch = mini_batch[0].to(device)
        augmented_batch = []  # To store 2N images
        for image in mini_batch:
            augmented_1 = preprocess_for_train(image, height=img_size, width=img_size, color_distort=True, crop=True, flip=True)
            augmented_2 = preprocess_for_train(image, height=img_size, width=img_size, color_distort=True, crop=True, flip=True)
            augmented_batch.extend([augmented_1, augmented_2])

        augmented_batch = torch.stack(augmented_batch)  # Shape: [2N, C, H, W]
        representations = model(augmented_batch.permute(0,3,1,2))
        projections = normalizer(representations, dim=1)
        loss = criterion(projections)
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        
    if scheduler:
        scheduler.step()

    tracker[epoch+1] = {"Loss": np.mean(losses), "lr": optimizer.param_groups[0]['lr']}
    print(f"Epoch {epoch+1}, Loss: {np.mean(losses)}, lr: {optimizer.param_groups[0]['lr']}")

    # Clear GPU memory
    del mini_batch, augmented_batch, representations, projections, loss
    if device == "cuda":
        torch.cuda.empty_cache()

    return losses

In [6]:
from copy import deepcopy

def eval(dataloader_val, model, device, normalizer, criterion, epoch, tracker, img_size):
    """Evaluate the model on the validation dataset."""
    eval_model = deepcopy(model)
    eval_model.eval()
    val_losses = []
    with torch.no_grad():
        for mini_batch in tqdm(dataloader_val, desc="Evaluating"):
            mini_batch = mini_batch[0].to(device)
            augmented_batch = []
            for image in mini_batch:
                augmented_1 = preprocess_for_train(image, height=img_size, width=img_size, color_distort=True, crop=True, flip=True)
                augmented_2 = preprocess_for_train(image, height=img_size, width=img_size, color_distort=True, crop=True, flip=True)
                augmented_batch.extend([augmented_1, augmented_2])
            augmented_batch = torch.stack(augmented_batch)
            representations = eval_model(augmented_batch.permute(0, 3, 1, 2))
            projections = normalizer(representations, dim=1)
            loss = criterion(projections)
            val_losses.append(loss.item())
            # Clear GPU memory for each batch
            del mini_batch, augmented_batch, representations, projections, loss
            if device == "cuda":
                torch.cuda.empty_cache()
    avg_loss = np.mean(val_losses)
    tracker[epoch+1]["Loss_val"] = avg_loss
    print(f"Epoch {epoch+1}, Validation Loss: {avg_loss:.4f}")
    del eval_model
    return val_losses

In [7]:
import json
import os

def save_checkpoint(model, optimizer, scheduler, total_loss, epoch, checkpoint_dir):
    checkpoint = {
        "epoch": epoch + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict() if scheduler else None,
        "total_loss": total_loss,
    }
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}.pt")
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")

In [8]:
from torch.nn.functional import cross_entropy

def l2_normalize(tensor, dim=1, epsilon=1e-12):
    """
    Performs L2 normalization on the given tensor along the specified dimension.
    
    Args:
        tensor (torch.Tensor): The input tensor to normalize.
        dim (int): The dimension along which to normalize (default: 1).
        epsilon (float): Small constant to avoid division by zero (default: 1e-12).
    
    Returns:
        torch.Tensor: L2-normalized tensor.
    """
    norm = torch.sqrt(torch.sum(tensor**2, dim=dim, keepdim=True) + epsilon)
    return tensor / norm

def contrastive_loss(projections, temperature=0.1):
    """
    Compute NT-Xent loss for contrastive learning.
    
    Args:
        projections (torch.Tensor): Tensor of shape [2N, d], where 2N is the batch size
                                    (N samples with two augmented views each) and d is
                                    the feature dimension.
        temperature (float): Temperature scaling parameter for the loss.
    
    Returns:
        torch.Tensor: Scalar loss value.
    """
    # Normalize projections to unit vectors
    # projections = F.normalize(projections, dim=1)

    # Compute similarity matrix (cosine similarity)
    similarity_matrix = torch.matmul(projections, projections.T)  # Shape: [2N, 2N]
    
    # Scale by temperature
    similarity_matrix = similarity_matrix / temperature

    # Create labels for positive pairs
    N = projections.size(0) // 2  # Number of original samples
    labels = torch.add(torch.arange(2*N, device=projections.device), torch.tensor([1, -1], device=projections.device).repeat(N))

    # Mask out self-similarities
    mask = torch.eye(2 * N, device=projections.device).bool()
    similarity_matrix.masked_fill_(mask, float('-inf'))

    # Compute loss using cross-entropy
    loss = cross_entropy(similarity_matrix, labels)
    return loss

In [9]:
from torch.optim.lr_scheduler import StepLR

lr=0.01
weight_decay=0.0001
gamma=0.5
step_size = 40

normalizer = l2_normalize
criterion = contrastive_loss

optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

tracker = {}

def train(model = model, dataloader = data_loader, dataloader_val = data_loader_val, device = "cpu", 
          img_size = img_size, normalizer = normalizer, criterion = criterion, optimizer = optimizer,
          tracker = tracker, scheduler = scheduler, num_epochs = 100, validationstep = 10,
          save_checkpoints = 10, checkpoint_dir = "checkpoints"):
    os.makedirs(checkpoint_dir, exist_ok=True)
    model.to(device)
    total_loss = []
    total_eval_loss = []
    for epoch in range(num_epochs):
        losses = train_one_epoch(model, dataloader, device, img_size, normalizer, criterion, optimizer, epoch, tracker, scheduler)
        total_loss.extend(losses)
        if (epoch+1)%validationstep == 0:
            eval_loss = eval(dataloader_val, model, device, normalizer, criterion, epoch, tracker, img_size)
            total_eval_loss.extend(list(eval_loss))
        if (epoch+1)%save_checkpoints == 0:
            save_checkpoint(model, optimizer, scheduler, total_loss, epoch, checkpoint_dir)
        if device == "cuda":
            torch.cuda.empty_cache()
    
    tracker_path = os.path.join(checkpoint_dir, "tracker.json")
    with open(tracker_path, "w") as f:
        json.dump(tracker, f)
    print(f"Tracker saved to {tracker_path}")

In [10]:
train(device = "cuda")

100%|██████████| 32/32 [00:03<00:00,  9.52it/s]


Epoch 1, Loss: 3.967508904635906, lr: 0.01


100%|██████████| 32/32 [00:02<00:00, 10.74it/s]


Epoch 2, Loss: 3.8524583876132965, lr: 0.01


100%|██████████| 32/32 [00:02<00:00, 10.73it/s]


...

Epoch 99, Loss: 1.0666022861842066, lr: 0.0025


100%|██████████| 32/32 [00:02<00:00, 10.69it/s]


Epoch 100, Loss: 1.0970654445700347, lr: 0.0025


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 12.79it/s]


Epoch 100, Validation Loss: 2.4960
Checkpoint saved: checkpoints/checkpoint_epoch_100.pt
Tracker saved to checkpoints/tracker.json
